In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt
#%matplotlib inline

def cal_chessBoard(images, nx, ny):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:ny,0:nx].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images


    # Step through the list and search for chessboard corners
    for fname in images:
        #print("test")
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (ny,nx),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            #img = cv2.drawChessboardCorners(img, (ny,nx), corners, ret)
            #cv2.imshow('img',img)
            #cv2.waitKey(500)

    #cv2.destroyAllWindows()
    return objpoints, imgpoints

def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist

def corners_unwarp(img, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped

def edge_detection(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])# the max value index of left side
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint# the max value index of left side

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    #print("nonzero: ", nonzero)
    nonzeroy = np.array(nonzero[0])
    #print("nonzeroy: ", nonzeroy)
    nonzerox = np.array(nonzero[1])
    #print("nonzerox: ", nonzerox)
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###

        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

        #pass # Remove this when you add your function

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]
    #out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return left_fit, right_fit

def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    #print("left_fit: ",left_fit)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2+left_fit[1]*ploty+left_fit[2]
    right_fitx = right_fit[0]*ploty**2+right_fit[1]*ploty+right_fit[2]
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit

def search_around_poly(binary_warped, left_fit, right_fit):

    margin = 100
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Use previous polynomials
    left_lane_inds = (nonzerox>left_fit[0]*nonzeroy**2+left_fit[1]*nonzeroy+left_fit[2]-margin)&(nonzerox<left_fit[0]*nonzeroy**2+left_fit[1]*nonzeroy+left_fit[2]+margin)
    right_lane_inds = (nonzerox>right_fit[0]*nonzeroy**2+right_fit[1]*nonzeroy+right_fit[2]-margin)&(nonzerox<right_fit[0]*nonzeroy**2+right_fit[1]*nonzeroy+right_fit[2]+margin)

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty, left_fit, right_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    return ploty, left_fitx, right_fitx, left_fit, right_fit

def measure_curvature_pixels(ploty, left_fit, right_fit):
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = (1+(2*left_fit[0]*y_eval+left_fit[1])**2)**1.5/abs(2*left_fit[0])  ## Implement the calculation of the left line here
    right_curverad = (1+(2*right_fit[0]*y_eval+right_fit[1])**2)**1.5/abs(2*right_fit[0])  ## Implement the calculation of the right line here
    
    return left_curverad, right_curverad

def draw_lanes(warped, undist, left_fitx, right_fitx, ploty, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        self.recent_fit = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = []  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 
        self.space = None

        
image = cv2.imread('test_images/test2.jpg')
#plt.imshow(image)
#print("image.shape: ",image.shape)
images = glob.glob('camera_cal/calibration*.jpg')
objpoints, imgpoints = cal_chessBoard(images, 6, 9)
mtx, dist = cal_undistort(image, objpoints, imgpoints)
src = np.float32([[550,450],[730,450],[0,670],[1280,670]])
dst = np.float32([[0,0],[1280,0],[0,720],[1280,720]])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)



In [2]:
left_lane=Line()
right_lane=Line()
fail_times=0
current_frame=0
lane_space=[]

def process_image(image):
    global left_lane, right_lane, fail_times,current_frame
    #print("current_frame: ",current_frame)
    current_frame+=1
    #resize image to 1280*720(X*Y)
    if image.shape[1]!=1280 or image.shape[0]!=720:
        resized_img = cv2.resize(image, (1280, 720))
    else:
        resized_img = image
    #undisortion
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    #edge detection
    edge = edge_detection(undist)
    #perspective transform
    warped = corners_unwarp(edge, src, dst)
    #plt.imshow(warped)
    if len(left_lane.best_fit)!=0 and len(right_lane.best_fit)!=0:# if lane line detected
        #print("position A")
        try:
            left_lane.ally, left_lane.allx, right_lane.allx, left_lane.current_fit, right_lane.current_fit = search_around_poly(warped, left_lane.best_fit, right_lane.best_fit)
            #update .detected
            #print("position A.1")

            left_lane.detected = True
            right_lane.detected = True
        except:
            #print("position A.2")
            fail_times+=1
            left_lane.detected = False
            right_lane.detected = False
        if fail_times>=5:#use sliding window method:
            #print("position A.3")
            try:
                left_fit, right_fit = fit_polynomial(warped)
                left_lane.best_fit = []
                right_lane.best_fit = []
                left_lane.recent_fit = []
                right_lane.recent_fit = []
                left_lane.ally, left_lane.allx, right_lane.allx, left_lane.current_fit, right_lane.current_fit = search_around_poly(warped, left_fit, right_fit)
                #update .detected
                #print("position A.4")

                left_lane.detected = True
                right_lane.detected = True
            except:
                #print("position A.5")
                #print("max fail_times reached, but lanes still not detected")
                return None
    else:#if lane line not detected:
        #print("position B")
        try:
            left_fit, right_fit = fit_polynomial(warped)
            left_lane.best_fit = []
            right_lane.best_fit = []
            left_lane.recent_fit = []
            right_lane.recent_fit = []
            left_lane.ally, left_lane.allx, right_lane.allx, left_lane.current_fit, right_lane.current_fit = search_around_poly(warped, left_fit, right_fit)
            #update .detected
            #print("position B.1")

            left_lane.detected = True
            right_lane.detected = True
        except:
            #update .detected
            #print("position B.2")
            fail_times+=1
            left_lane.detected = False
            right_lane.detected = False
            if fail_times>=5:#if fail from the beginning and reached max fail_times
                #print("position B.3")
                #print("max fail_times reached, but lanes still not detected")
                return None
    
    if left_lane.detected and right_lane.detected:# if lane line detected in latest iteration
        #print("position C")
        if len(left_lane.best_fit)!=0 and len(right_lane.best_fit)!=0:
            left_lane.diffs = np.mean(abs(left_lane.current_fit - left_lane.best_fit))
            right_lane.diffs = np.mean(abs(right_lane.current_fit - right_lane.best_fit))
            left_lane.space = np.max(right_lane.allx - left_lane.allx)-np.min(right_lane.allx - left_lane.allx)
            lane_space.append(left_lane.space)
        else:
            left_lane.diffs = 0.0
            right_lane.diffs = 0.0
            left_lane.space = 0.0
        #print("left_lane.diffs: ",left_lane.diffs)
        #print("right_lane.diffs: ",right_lane.diffs)
        #print("left_lane.parallel: ",left_lane.parallel)
        
        #sanity check defluctuation(remove sudden change and inparallel):
        if left_lane.diffs<60 and right_lane.diffs<60 and left_lane.space<200:#to filter out strange fits
            #print("position C.1")
            #then to get/update best_fit:
            left_lane.recent_fit.append(left_lane.current_fit)
            right_lane.recent_fit.append(right_lane.current_fit)

            if len(left_lane.recent_fit)==1:
                #print("position C.2")
                left_lane.best_fit=left_lane.current_fit
                #print("len(left_lane.best_fit): ",len(left_lane.best_fit))
                #print("left_lane.best_fit: ",left_lane.best_fit)
                right_lane.best_fit=right_lane.current_fit
                #print("len(right_lane.best_fit): ",len(right_lane.best_fit))
                #print("right_lane.best_fit: ",right_lane.best_fit)
            else:
                #print("position C.3")
                if len(left_lane.recent_fit)==4:
                    #print("position C.4")
                    left_lane.recent_fit.pop(0)
                    right_lane.recent_fit.pop(0)
                left_lane.best_fit=np.mean(left_lane.recent_fit,0)
                right_lane.best_fit=np.mean(right_lane.recent_fit,0)
            left_lane.bestx = left_lane.allx
            right_lane.bestx = right_lane.allx
            fail_times=0 #reset
        else:
            fail_times+=1
            left_lane.detected = False
            right_lane.detected = False
        #print("position D")
        left_lane.radius_of_curvature, right_lane.radius_of_curvature = measure_curvature_pixels(left_lane.ally, left_lane.best_fit, right_lane.best_fit)
    # if sanity check failed, then use last time lane lines. but if max fail_times reached, then use silding window method
    image_output = draw_lanes(warped, image, left_lane.bestx, right_lane.bestx, left_lane.ally, Minv)
    #print("position E")
    return image_output

#image_output = process_image(image)
#plt.imshow(image_output)

In [3]:
white_output = 'project_video_out.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
for item in lane_space:
    print(item)

t:   0%|          | 2/1260 [00:00<01:08, 18.26it/s, now=None]

Moviepy - Building video project_video_out.mp4.
Moviepy - Writing video project_video_out.mp4



Moviepy - Done !
Moviepy - video ready project_video_out.mp4
CPU times: user 3min 51s, sys: 34.5 s, total: 4min 26s
Wall time: 2min 27s
113.0228897273197
115.19216899083972
120.17321659534446
124.20675222685145
129.14714922519101
143.49930258117422
132.5589528941208
131.52602269311342
123.31435743717304
125.47783185195078
132.42262590763812
101.79428629316192
111.81250426337067
111.85878679470613
138.34764717525525
157.7179687950213
166.75540711129122
171.39319097672728
155.38514320877596
131.45085186582583
120.97249434480204
106.86782814324044
101.95883717924619
113.50055805128068
123.60233075222538
107.96289376925347
107.54248871270147
91.45443054994325
107.13898139148921
114.37883110777932
144.89938435634406
161.30709676188553
128.0038941384928
144.99942086633814
147.53790124412467
157.14181497500317
153.14807433162025
158.621091071597
170.48034364721752
167.20485630033477
172.92687696555868
169.12774848948015
162.22751482114109
143.29728264899995
130.2313432450252
120.0340300943095

In [4]:
HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(white_output))